<a href="https://colab.research.google.com/github/IA-Cardiologia-husa/Amylearning/blob/main/Calculator_AMYLEARNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ML-based Score for the diagnosis of Cardiac Amyloidosis**
## *from Echocardiographic, Electrocardiographic and (optionally) Automatic Strain Measurements*

This calculator computes the probability of patient to have developed ATTR cardiac amyloidosis detectable by Radionucleide Bone Scintigraphy.

To use this calculator, the patient data has to be introduced in the form below, and then press Ctrl+F9 or press Runtime -> Run All. The results will be shown at the bottom of the page.

If one wishes to use the Automatic Strain Measurements, a dialog for uploading a 4-chamber view echocardiogram in dicom format will appear. Once the upload is complete, the calculator will start to process it. This takes long, and depending on the number of frames of the echocardiogram, it can take between 1 and 2 hours of processing. An improved result will be shown afterwards.

**Disclaimer: This calculator has been developed for research and demonstration purposes and not for clinical application.**

In [ ]:
# @title Demographic Data

Sex = 'Man' #@param ['Man', 'Woman']
Age = 72 #@param {type:'slider', min:20, max:110}
Weight_kg = 73  #@param {type: "slider", min: 30, max: 180}
Height_cm = 164  #@param {type: "slider", min: 100, max: 250}

In [ ]:
# @title Echocardiographic Data

Septum_Wall_Thickness = 1.44  #@param {type: 'slider', min: 0.00, max: 4, step:0.01}
LV_Posterior_Wall_Thickness = 1.10  #@param {type: 'slider', min: 0.00, max: 4, step:0.01}
LV_Diameter = 4.54  #@param {type: 'slider', min: 0.00, max: 8, step:0.01}
LV_telediastolic_volume = 90  #@param {type: 'slider', min: 0, max: 350, step:1}
LV_telesystolic_volume = 35  #@param {type: 'slider', min: 0, max: 280, step:1}
Myocardium_Mass = 229  #@param {type: 'slider', min: 0, max: 700, step:1}
LA_Volume = 77  #@param {type: 'slider', min: 0, max: 300, step:1}
TissueDoppler_e_wave_peak_velocity_median = 5.22  #@param {type: 'slider', min: 0.00, max: 20, step:0.01}
TissueDoppler_e_wave_peak_velocity_lateral = 6.92  #@param {type: 'slider', min: 0.00, max: 30, step:0.01}
TissueDoppler_eeprime_ratio_lateral = 11  #@param {type: 'slider', min: 0, max: 70, step:1}
TissueDoppler_eeprime_ratio_average = 12  #@param {type: 'slider', min: 0, max: 70, step:1}
LVOT_avg_velocity = 63  #@param {type: 'slider', min: 0, max: 240, step:1}
LVOT_avg_pressure_gradient = 1.91  #@param {type: 'slider', min: 0.00, max: 24, step:0.01}
LVOT_Velocity_Time_Integral = 19  #@param {type: 'slider', min: 0, max: 90, step:1}
AorticValve_peak_velocity = 150  #@param {type: 'slider', min: 0, max: 700, step:1}
AorticValve_peak_pressure_gradient = 9  #@param {type: 'slider', min: 0, max: 160, step:1}
AorticValve_mean_velocity = 103  #@param {type: 'slider', min: 0, max: 550, step:1}
AorticValve_mean_pressure_gradient = 4  #@param {type: 'slider', min: 0, max: 100, step:1}
AorticValve_Velocity_Time_Integral = 30  #@param {type: 'slider', min: 0, max: 180, step:1}
Pulmonary_Acceleration_Time = 0.10  #@param {type: 'slider', min: 0.00, max: 0.3, step:0.01}
Pulmonary_Acceleration_Slope = 753  #@param {type: 'slider', min: 0, max: 4000, step:1}

In [ ]:
# @title Electrodiographic Data

Sinusal_rhythm = True  #@param {type: 'boolean'}
AV_block = False  #@param {type: 'boolean'}
PR_Interval_ms = 126  #@param {type: 'slider', min: 0, max: 500, step:1}
QRS_angle = 0  #@param {type: 'slider', min: -180, max: 180, step:15}
QRS_duration_ms = 100  #@param {type: 'slider', min: 0, max: 236, step:1}
RR_interval_ms = 857  #@param {type: 'slider', min: 300, max: 1900, step:1}
QT_interval_ms = 410  #@param {type: 'slider', min: 250, max: 650, step:1}
Pwave_duration_ms = 40  #@param {type: 'slider', min: 0, max: 160, step:1}
Mitral_P_wave = False  #@param {type: 'boolean'}
Pulmonar_P_wave = False  #@param {type: 'boolean'}
IntraAtrial_Block = False  #@param {type: 'boolean'}
Left_Bundle_Branch_Block = False  #@param {type: 'boolean'}
Right_Bundle_Branch_Block = False  #@param {type: 'boolean'}
Hemiblock = False  #@param {type: 'boolean'}
Other_Intraventricular_Block = False  #@param {type: 'boolean'}
QRS_notch = False  #@param {type: 'boolean'}
LV_Hypertrophy_Criteria = False  #@param {type: 'boolean'}
Systolic_Overload = False  #@param {type: 'boolean'}
Q_Wave = False  #@param {type: 'boolean'}
Negative_T = False  #@param {type: 'boolean'}
ST_depression = False  #@param {type: 'boolean'}
Pseudoinfarction_pattern_V1_V2_V3 = False  #@param {type: 'boolean'}


In [ ]:
# @title Prediction with Echocardiographic and Electrocardiographic data only
import datetime as dt
import pandas as pd
import numpy as np
import xgboost
import os

from IPython.utils import io as ip_io
from IPython.display import HTML, Markdown
from base64 import b64encode
from google.colab import files

# Code for downloading the models
!gdown '158XWuLQgCHN0_rM05QwnNBG31R-EXXlT'
model = xgboost.XGBClassifier()
model.load_model('XGB_AmyloidosisConEdadSinStrain.sav')


echo_variables = ['Peso, kg',	'Talla, cm',	'ASC',	'IMC',	'SIVtd',	'DVItd',
			'PPVItd',	'VTD_combinado',	'VTDVI_index',	'VTS_combinado',	'VLatidoVI_combinado',
			'FEVI_combinado',	'MasaVI_cubico',	'MasaVI_index',	'GrosorRelativoPared',
			'VolAI_combinado',	'VolAIindex_comb',	'Vel.Epicomed',	'Vel.Epicolat',
			'Eelat', 'Eeprom',	'V1mediaVI',	'GPmedV1VI',	'ITVV1VI',	'VmáxV2Ao',
			'GPmáxAo',	'V2mediaAo',	'GPmedAo', 'ITVV2Ao',	'TiempoacelAP',	'PendienteacelAP']

ekg_variables = ['ECG ritmo',
			'Bloqueo auriculoventricular', 'ECG frecuencia cardiaca',
			'Intervalo PR, msg', 'Ángulo QRS', 'Anchura QRS, msg',
			'Intervalo RR, msg', 'Intervalo QT, msg', 'QT corregido (Bazett), msg',
			'Anchura onda P, msg', 'Onda P mitral', 'Onda P pulmonar',
			'Bloqueo intraauricular', 'Bloqueo rama izquierda', 'Bloqueo rama derecha',
			'Transtorno de la conducción intraventricular',
			'Empastamientos (notch) QRS', 'Hemibloqueo', 'Criterios HVI',
			'Sobrecarga sistólica', 'Onda Q', 'T negativa', 'Descenso ST',
			'Patrón pseudoinfarto V1-3']


df = pd.DataFrame(columns = echo_variables+ekg_variables+['Sexo\n0=H; 1=M','Edad'])


df['Sexo\n0=H; 1=M'] = [0 if Sex == 'Man' else 1]
df['Edad'] = [Age]
df['Peso, kg'] = [Weight_kg]
df['Talla, cm'] = [Height_cm]
df['ASC'] =  np.sqrt(df['Talla, cm'] * df['Peso, kg'] / 3600)
df['IMC'] = 10000*df['Peso, kg'] / df['Talla, cm']**2

df['SIVtd'] = [Septum_Wall_Thickness]
df['DVItd'] = [LV_Diameter ]
df['PPVItd'] = [LV_Posterior_Wall_Thickness ]
df['VTD_combinado'] = [LV_telediastolic_volume ]
df['VTDVI_index'] = df['VTD_combinado']/df['ASC']
df['VTS_combinado'] = [LV_telesystolic_volume]
df['VLatidoVI_combinado'] = df['VTD_combinado']-df['VTS_combinado']
df['FEVI_combinado'] = 100*(df['VTD_combinado']-df['VTS_combinado'])/df['VTD_combinado']
df['MasaVI_cubico'] = [Myocardium_Mass]
df['MasaVI_index'] = df['MasaVI_cubico']/df['ASC']
df['GrosorRelativoPared'] = 2*df['PPVItd']/df['DVItd']
df['VolAI_combinado'] = [LA_Volume]
df['VolAIindex_comb'] = df['VolAI_combinado']/df['ASC']
df['Vel.Epicomed'] = [TissueDoppler_e_wave_peak_velocity_median]
df['Vel.Epicolat'] = [TissueDoppler_e_wave_peak_velocity_lateral]
df['Eelat'] = [TissueDoppler_eeprime_ratio_lateral]
df['Eeprom'] = [TissueDoppler_eeprime_ratio_average]
df['V1mediaVI'] = [LVOT_avg_velocity]
df['GPmedV1VI'] = [LVOT_avg_pressure_gradient]
df['ITVV1VI'] = [LVOT_Velocity_Time_Integral]
df['VmáxV2Ao'] = [AorticValve_peak_velocity ]
df['GPmáxAo'] = [AorticValve_peak_pressure_gradient]
df['V2mediaAo'] = [AorticValve_mean_velocity ]
df['GPmedAo'] = [AorticValve_mean_pressure_gradient]
df['ITVV2Ao'] = [AorticValve_Velocity_Time_Integral ]
df['TiempoacelAP'] = [Pulmonary_Acceleration_Time ]
df['PendienteacelAP'] = Pulmonary_Acceleration_Slope

df['ECG ritmo'] = [1 if Sinusal_rhythm else 2]
df['Bloqueo auriculoventricular'] = [1 if AV_block else 0]
df['Intervalo RR, msg'] = [RR_interval_ms]
df['ECG frecuencia cardiaca'] = 60000/df['Intervalo RR, msg']
df['Intervalo PR, msg'] = [PR_Interval_ms]
df['Ángulo QRS'] = [QRS_angle]
df['Anchura QRS, msg'] = [QRS_duration_ms]
df['Intervalo QT, msg'] = [QT_interval_ms]
df['QT corregido (Bazett), msg'] = df['Intervalo QT, msg'] / np.sqrt(df['Intervalo RR, msg']/1000.)
df['Anchura onda P, msg'] = [Pwave_duration_ms]
df['Onda P mitral'] = [1 if Mitral_P_wave else 0]
df['Onda P pulmonar'] = [1 if Pulmonar_P_wave else 0]
df['Bloqueo intraauricular'] = [1 if IntraAtrial_Block else 0]
df['Bloqueo rama izquierda'] = [1 if Left_Bundle_Branch_Block else 0]
df['Bloqueo rama derecha'] = [1 if Right_Bundle_Branch_Block else 0]
df['Transtorno de la conducción intraventricular'] = [1 if Other_Intraventricular_Block else 0]
df['Empastamientos (notch) QRS'] = [1 if QRS_notch else 0]
df['Hemibloqueo'] = [1 if Hemiblock else 0]
df['Criterios HVI'] = [1 if LV_Hypertrophy_Criteria else 0]
df['Sobrecarga sistólica'] = [1 if Systolic_Overload else 0]
df['Onda Q'] = [1 if Q_Wave else 0]
df['T negativa'] = [1 if Negative_T else 0]
df['Descenso ST'] = [1 if ST_depression else 0]
df['Patrón pseudoinfarto V1-3'] = [1 if Pseudoinfarction_pattern_V1_V2_V3 else 0]

prob = model.predict_proba(df.loc[:, ['Sexo\n0=H; 1=M','Edad'] + echo_variables + ekg_variables])[0,1]

display(Markdown(f'# Results \n The estimated probability of this patient to have cardiac amyloidosis is {100*prob:.2f}%'))


In [ ]:
# @title Prediction with automated strain information from an uploaded 4-chamber view echocardiogram
import os

uploaded = files.upload()
filename = list(uploaded.keys())[0]
try:
  os.makedirs('./dcms')
except:
  pass
if filename[-4:] == '.dcm':
  os.rename(filename, './dcms/'+filename)
else:
  os.rename(filename, './dcms/'+filename+'.dcm')

tt = dt.datetime.now()
print("(1/8) Downloading necessary packages")
with ip_io.capture_output() as captured:
  !pip install luigi
  !pip install pydicom
  !pip install SimpleITK
  !git clone https://github.com/IA-Cardiologia-HUSA/Amylearning
  from Amylearning.AutoECHO import *
  !gdown '109Z2wYuOIlbiDjcnaFpAHd_QfroOaeNc'
  !gdown '1YRXCspE3KCOjJEfQyPgngE_bB-0uhhi0'
t0 = dt.datetime.now()
print(t0-tt)

print("(2/8) Segmenting LV")
luigi.build([SegmentVentricleDCM(dcm_name = filename[:-4])], workers=2, local_scheduler=True, log_level='WARNING')
t1 = dt.datetime.now()
print(t1-t0)

print("(3/8) Identifying reliable, end-systolic, and end-diastolic frames")
with ip_io.capture_output() as captured2:
  luigi.build([VolumeCurvesDCM(dcm_name=filename[:-4])], workers=2, local_scheduler=True)
t2 = dt.datetime.now()
print(t2-t1)

print("(4/8) Creating video for reviewing the segmentation")
with ip_io.capture_output() as captured3:
  luigi.build([VideoSegmentationDCM(dcm_name=filename[:-4], myo_segmentation=True)], workers=2, local_scheduler=True)
t3 = dt.datetime.now()
print(t3-t2)

os.system(f"ffmpeg -i '/content/reports/{filename[:-4]}/VideoSegmentationDCM/segmentation_myo.mp4' -vcodec libx264 '/content/reports/{filename[:-4]}/VideoSegmentationDCM/segmentation_myo2.mp4'")
mp4 = open(f'/content/reports/{filename[:-4]}/VideoSegmentationDCM/segmentation_myo2.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
display(HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url))

print("(5/8) Calculating differences between frames")
with ip_io.capture_output() as captured4:
  luigi.build([SumSquaredDifferencesDCM(dcm_name = filename[:-4], window_halfsize = 6, region_halfsize =4)], workers=2, local_scheduler=True)
t4 = dt.datetime.now()
print(t4-t3)

print("(6/8) Tracking myocardium movement")
with ip_io.capture_output() as captured5:
  luigi.build([TrackDCM(dcm_name = filename[:-4])], workers=2, local_scheduler=True)
t5 = dt.datetime.now()
print(t5-t4)


print("(7/8) Calculating strain")
with ip_io.capture_output() as captured6:
  luigi.build([VideoTrackingDCM(dcm_name=filename[:-4])], workers=2, local_scheduler=True)
t6 = dt.datetime.now()
print(t6-t5)

os.system(f"ffmpeg -i '/content/reports/{filename[:-4]}/VideoTrackingDCM/strain_field.mp4' -vcodec libx264 '/content/reports/{filename[:-4]}/VideoTrackingDCM/strain_field2.mp4'")
mp4_b = open(f'/content/reports/{filename[:-4]}/VideoTrackingDCM/strain_field2.mp4','rb').read()
data_url_b = "data:video/mp4;base64," + b64encode(mp4_b).decode()
display(HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url_b))

print("(8/8) Gathering strain and volumetric measurements")
with ip_io.capture_output() as captured7:
  luigi.build([MeasurementsVolumesDCM(dcm_name=filename[:-4]), MeasurementsStrainDCM(dcm_name=filename[:-4])], workers=2, local_scheduler=True)
t7 = dt.datetime.now()
print(t7-t6)


df_strain = pd.read_csv(f'./reports/{filename[:-4]}/MeasurementsStrainDCM/measurements_strain.csv')
df_volume = pd.read_csv(f'./reports/{filename[:-4]}/MeasurementsVolumesDCM/measurements_volume.csv')
df_total = pd.concat([df, df_volume,df_strain], axis = 'columns')

model_strain = xgboost.XGBClassifier()
model_strain.load_model('XGB_AmyloidosisConEdadConStrain.sav')

strain_long_variables= ['gls_peak_contraction',
      'gls_peak_distention',
      'gls_peak_contraction_rate',
      'gls_peak_distention_rate',
      'rls0_peak_contraction',
      'rls0_peak_distention',
      'rls0_peak_contraction_rate',
      'rls0_peak_distention_rate',
      'rls1_peak_contraction',
      'rls1_peak_distention',
      'rls1_peak_contraction_rate',
      'rls1_peak_distention_rate',
      'rls2_peak_contraction',
      'rls2_peak_distention',
      'rls2_peak_contraction_rate',
      'rls2_peak_distention_rate',
      'rls3_peak_contraction',
      'rls3_peak_distention',
      'rls3_peak_contraction_rate',
      'rls3_peak_distention_rate',
      'rls4_peak_contraction',
      'rls4_peak_distention',
      'rls4_peak_contraction_rate',
      'rls4_peak_distention_rate',
      'rls5_peak_contraction',
      'rls5_peak_distention',
      'rls5_peak_contraction_rate',
      'rls5_peak_distention_rate']

strain_trans_variables=['gts_peak_contraction',
      'gts_peak_distention',
      'gts_peak_contraction_rate',
      'gts_peak_distention_rate',
      'rts0_peak_contraction',
      'rts0_peak_distention',
      'rts0_peak_contraction_rate',
      'rts0_peak_distention_rate',
      'rts1_peak_contraction',
      'rts1_peak_distention',
      'rts1_peak_contraction_rate',
      'rts1_peak_distention_rate',
      'rts2_peak_contraction',
      'rts2_peak_distention',
      'rts2_peak_contraction_rate',
      'rts2_peak_distention_rate',
      'rts3_peak_contraction',
      'rts3_peak_distention',
      'rts3_peak_contraction_rate',
      'rts3_peak_distention_rate',
      'rts4_peak_contraction',
      'rts4_peak_distention',
      'rts4_peak_contraction_rate',
      'rts4_peak_distention_rate',
      'rts5_peak_contraction',
      'rts5_peak_distention',
      'rts5_peak_contraction_rate',
      'rts5_peak_distention_rate']
automatic_volumes = ['LVEF (automated)',
      'Dyastolic volume (automated)',
      'Systolic volume (automated)',
      'peak_vol_relative_contraction_rate',
      'peak_vol_relative_distention_rate',
      'peak_vol_absolute_contraction_rate',
      'peak_vol_absolute_distention_rate']

pred_strain = model_strain.predict_proba(df_total.loc[:, echo_variables+ekg_variables+strain_long_variables+strain_trans_variables+automatic_volumes+['Sexo\n0=H; 1=M','Edad']])[0,1]

display(Markdown(f'# Results \n Using the strain information, the estimated probability of this patient to have cardiad amyloidosis is {100*pred_strain:.2f}%'))